In [1]:
#%env THEANO_FLAGS=device=gpu7
import numpy as np
import theano
import theano.tensor as T
import lasagne
import os
#thanks @keskarnitish

env: THEANO_FLAGS=device=gpu7


WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 7: GeForce GTX 1080 (CNMeM is enabled with initial size: 45.0% of memory, cuDNN 5105)


# Questions prediction

* Here we try to generate human-readable questions.
* The dataset origins from https://www.kaggle.com/c/quora-question-pairs - the challanged started recently about finidng duplicated questions.
* You should use train (not test!) set>

In [13]:
import pandas as pd
df = pd.read_csv("./train.csv")

In [14]:
df = pd.concat([df.question1,df.question2])

In [15]:
questions = list(set(df))

In [16]:
questions = filter(lambda x: type(x) is str, questions)

In [17]:
start_token,end_token = " ",";"
questions = [start_token+name.replace(";",",")+end_token for name in questions]

In [18]:
len(questions)

537361

In [19]:
questions[0]

" How do I open petrol pump in India? What's the procedure?;"

In [20]:
print('n samples = ',len(questions))
for x in questions[::100000]:
    print (x)

n samples =  537361
 How do I open petrol pump in India? What's the procedure?;
 How do you come to know that you love her or not?;
 What’s the least expensive way to fly business class?;
 When was the last time you played a video game?;
 How do reciprocating pumps work?;
 What happens when you call 1-800-USA-RAIL?;


# Text processing

In [21]:
#all unique characters go here
token_set = set()
for name in questions:
    for letter in name:
        token_set.add(letter)

tokens = list(token_set)

#print 'n_tokens = ',len(tokens)


In [22]:
#!token_to_id = <dictionary of symbol -> its identifier (index in tokens list)>
token_to_id = {t:i for i,t in enumerate(tokens) }

#!id_to_token = < dictionary of symbol identifier -> symbol itself>
id_to_token = {i:t for i,t in enumerate(tokens)}

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.hist(list(map(len,questions)),bins=25);

# Truncate questions longer than MAX_LEN characters. 
MAX_LEN = min([150,max(list(map(len,questions)))])

NameError: name 'questions' is not defined

### Cast everything from symbols into identifiers

In [24]:
questions_ix = list(map(lambda name: list(map(token_to_id.get,name)),questions))


#crop long questions and pad short ones
for i in range(len(questions_ix)):
    questions_ix[i] = questions_ix[i][:MAX_LEN] #crop too long
    
    if len(questions_ix[i]) < MAX_LEN:
        questions_ix[i] += [token_to_id[" "]]*(MAX_LEN - len(questions_ix[i])) #pad too short
        
assert len(set(map(len,questions_ix)))==1

questions_ix = np.array(questions_ix)

# Input variables

In [25]:
input_sequence = T.matrix('token sequencea','int32')
target_values = T.matrix('actual next token','int32')


# Build NN

You will be building a model that takes token sequence and predicts next token


* iput sequence
* one-hot / embedding
* recurrent layer(s)
* otput layer(s) that predict output probabilities


In [26]:
from lasagne.layers import InputLayer,DenseLayer,EmbeddingLayer
from lasagne.layers import RecurrentLayer,LSTMLayer,GRULayer,CustomRecurrentLayer

In [28]:
l_in = lasagne.layers.InputLayer(shape=(None, None),input_var=input_sequence)

#!<Your neural network>
l_emb = <embedding layer or one-hot encoding>

l_rnn = <some recurrent layer(or several such layers)>

#flatten batch and time to be compatible with feedforward layers (will un-flatten later)
l_rnn_flat = lasagne.layers.reshape(l_rnn, (-1,l_rnn.output_shape[-1]))

l_out = <last dense layer (or several layers), returning probabilities for all possible next tokens>


In [29]:
# Model weights
weights = lasagne.layers.get_all_params(l_out,trainable=True)
print(weights)

[W, W_in_to_ingate, W_hid_to_ingate, b_ingate, W_in_to_forgetgate, W_hid_to_forgetgate, b_forgetgate, W_in_to_cell, W_hid_to_cell, b_cell, W_in_to_outgate, W_hid_to_outgate, b_outgate, W_cell_to_ingate, W_cell_to_forgetgate, W_cell_to_outgate, W, b]


In [30]:
network_output = lasagne.layers.get_output(l_out)
#If you use dropout do not forget to create deterministic version for evaluation

In [31]:

predicted_probabilities_flat = network_output
correct_answers_flat = target_values.ravel()


loss = <Loss function - a simple categorical crossentropy will do, maybe add some regularizer>

updates = <your favorite optimizer>


# Compiling it

In [32]:

#training
train = theano.function([input_sequence, target_values], loss, updates=updates, allow_input_downcast=True)

#computing loss without training
compute_cost = theano.function([input_sequence, target_values], loss, allow_input_downcast=True)


# Generation

Simple: 
* get initial context(seed), 
* predict next token probabilities,
* sample next token, 
* add it to the context
* repeat from step 2

You'll get a more detailed info on how it works in the homework section.

In [33]:
#compile the function that computes probabilities for next token given previous text.

#reshape back into original shape
next_word_probas = network_output.reshape((input_sequence.shape[0],input_sequence.shape[1],len(tokens)))
#predictions for next tokens (after sequence end)
last_word_probas = next_word_probas[:,-1]
probs = theano.function([input_sequence],last_word_probas,allow_input_downcast=True)


In [34]:
def generate_sample(seed_phrase=None,N=MAX_LEN,t=1,n_snippets=1):
    '''
    The function generates text given a phrase of length at least SEQ_LENGTH.
        
    parameters:
        sample_fun - max_ or proportional_sample_fun or whatever else you implemented
        
        The phrase is set using the variable seed_phrase

        The optional input "N" is used to set the number of characters of text to predict.     
    '''
    if seed_phrase is None:
        seed_phrase=start_token
    if len(seed_phrase) > MAX_LEN:
        seed_phrase = seed_phrase[-MAX_LEN:]
    assert type(seed_phrase) is str

    
    snippets = []
    for _ in range(n_snippets):
        sample_ix = []
        x = list(map(lambda c: token_to_id.get(c,0), seed_phrase))
        x = np.array([x])

        for i in range(N):
            # Pick the character that got assigned the highest probability
            
            
            p = <find next to x char> 
            p = p**t / np.sum(p**t)
            
            ix = <choose random token number less than len(token) with p>
            
            sample_ix.append(ix)

            x = np.hstack((x[-MAX_LEN+1:],[[ix]]))

        random_snippet = seed_phrase + ''.join(id_to_token[ix] for ix in sample_ix)    
        snippets.append(random_snippet)
        
    print("----\n %s \n----" % '; '.join(snippets))


# Model training

Here you can tweak parameters or insert your generation function


__Once something word-like starts generating, try increasing seq_length__



In [35]:
def sample_batch(data, batch_size):
    
    rows = data[np.random.randint(0,len(data),size=batch_size)]
    
    return rows[:,:-1],rows[:,1:]

In [40]:

print("Training ...")


#total N iterations
n_epochs=100

# how many minibatches are there in the epoch 
batches_per_epoch = 500

#how many training sequences are processed in a single function call
batch_size=10


for epoch in range(n_epochs):

    print("Generated questions")
    generate_sample(n_snippets=10, t=2)

    avg_cost = 0;
    
    for _ in range(batches_per_epoch):
        
        x,y = sample_batch(questions_ix,batch_size)
        avg_cost += train(x, y)
        
    print("Epoch {} average loss = {}".format(epoch, avg_cost / batches_per_epoch))



Training ...
Generated names
----
  What is the Fire better you service in a projection engineer;  What are the best between the to be a phone of every solvers;  What is the most work of the polition of the port in the add;  What are the best way to be a good from an engineering?;    ;  Why is the world bound be with engineering a polling to have;  What is the best social course for a many start of the perso;  What are your fact to watch of the pain advantage better?;  ;  What is the best way to prepany and which compander better i;  What would happen if you think a service program like at be ;  What is the best way to start a should prepare to serving ac 
----
Epoch 0 average loss = 1.2233946222066878
Generated names
----
  Is there any correct place a can for a constart?;           ;  How can I make in India?;                                   ;  I'm in the right for a hard Dadement program?;              ;  What is it like to be a stront drow of the US movie of the o;  What is the f

KeyboardInterrupt: 

# Examples
* Why are there any block way to start a girl and why?
* What is the difference between a companies have a some companies?;
* ...

In [2]:
generate_sample(n_snippets=10,t=1.5)

NameError: name 'generate_sample' is not defined

In [49]:
generate_sample(seed_phrase=" What is the difference",n_snippets=10, t=2)

----
  What is the difference between a cheat the university without in the USA and State;  What is the difference between development and company in the United Rangal Day"?;;  What is the difference between a companies have a some companies?;                ;  What is the difference between the having the Parammets for an iPhone game ever an;  What is the difference between a photos and a battle correct files?;              ;  What is the difference between a single policy students in a student?;            ;  What is the difference between a particular state of questions on Quora?;         ;  What is the difference between a policy of the life and with a laptop and a senten;  What is the difference between a marents?;                                        ;  What is the difference between a career sex and can using in a successful?;        
----


# And now,
* Try several layers
* Try your own dataset of any kind